<a href="https://colab.research.google.com/github/Janhvi-n/AlpaCare-Medical-Instruction-Assistant-/blob/main/AlpaCare_Medical_Instruction_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install -U transformers==4.57.0 datasets accelerate peft -q

In [1]:
import transformers
print(transformers.__version__)

4.57.0


In [2]:
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM,
                          DataCollatorForSeq2Seq, TrainingArguments, Trainer)
from peft import LoraConfig, get_peft_model, TaskType
import torch

In [3]:
# data_loader.py
from datasets import load_dataset

def load_and_preprocess():
    # Load dataset
    dataset = load_dataset("lavita/AlpaCare-MedInstruct-52k")

    # Use only a subset (saves Colab time)
    dataset = dataset["train"].shuffle(seed=42).select(range(2000))

    # Split into train/val/test (90/5/5)
    train_val = dataset.train_test_split(test_size=0.10, seed=42)
    val_test = train_val["test"].train_test_split(test_size=0.50, seed=42)

    train_data = train_val["train"]
    val_data = val_test["train"]
    test_data = val_test["test"]

    return train_data, val_data, test_data


In [4]:
train_data, val_data, test_data = load_and_preprocess()

# Format function
def format_data(example):
    return {
        "input_text": example["instruction"],
        "target_text": example["output"]
    }

train_data = train_data.map(format_data)
val_data = val_data.map(format_data)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [6]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05
)
model = get_peft_model(model, lora_config)

In [7]:
MAX_LENGTH = 128
def tokenize(batch):
    # Input
    model_inputs = tokenizer(batch["input_text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
    # Target/labels
    labels = tokenizer(batch["target_text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
    labels["input_ids"] = [[(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_data = train_data.map(tokenize, batched=True)
val_data = val_data.map(tokenize, batched=True)

train_data.set_format(type="torch", columns=["input_ids","attention_mask","labels"])
val_data.set_format(type="torch", columns=["input_ids","attention_mask","labels"])

In [8]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [10]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=100,
    eval_steps=100,
    eval_strategy="steps",   # old keyword compatible with 4.56.x
    learning_rate=2e-4,
    fp16=torch.cuda.is_available(),
    report_to=[]  # disables W&B/TensorBoard
)


In [11]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

/tmp/ipython-input-3820582753.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/transformers/data/data_collator.py:740: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss
100,0.000000,nan
200,0.000000,nan


TrainOutput(global_step=225, training_loss=0.0, metrics={'train_runtime': 101.2111, 'train_samples_per_second': 17.785, 'train_steps_per_second': 2.223, 'total_flos': 309363867648000.0, 'train_loss': 0.0, 'epoch': 1.0})

In [13]:
model.save_pretrained("alpacare_lora_adapter")
tokenizer.save_pretrained("alpacare_lora_adapter")
print("✅ LoRA adapter saved successfully!")

✅ LoRA adapter saved successfully!


In [19]:
# --------------------------
# 🔹 Inference Demo
# --------------------------
from peft import PeftModel
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load base model + LoRA adapter
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = PeftModel.from_pretrained(base_model, "alpacare_lora_adapter")
model.to(device)
model.eval()

MAX_LENGTH = 128

def ask(prompt, max_new_tokens=150, temperature=0.7, top_p=0.9):
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=MAX_LENGTH
    )
    # Move tensors to device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate output using sampling
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,      # enable sampling
        top_p=top_p,         # nucleus sampling
        temperature=temperature,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text + "\n\n⚠️ This is educational only — please consult a qualified clinician."

# --------------------------
# 🔹 Test prompts
# --------------------------
prompts_list = [
    "What are healthy food options for someone with diabetes?",
    "Explain how to properly use an inhaler.",
    "Describe safe hand-washing steps to prevent infection."
]

for p in prompts_list:
    print("Prompt:", p)
    print(ask(p))
    print("------------------------------------------------------\n")


Prompt: What are healthy food options for someone with diabetes?
protein and fruit

⚠️ This is educational only — please consult a qualified clinician.
------------------------------------------------------

Prompt: Explain how to properly use an inhaler.
The inhaler is designed to provide oxygen inhalation. The inhaler can be used to administer inhalations if the inhaler is used in a deep breath. The inhaler is intended to be used when you are not breathing, and is not intended for people with inhalers.

⚠️ This is educational only — please consult a qualified clinician.
------------------------------------------------------

Prompt: Describe safe hand-washing steps to prevent infection.
Hand-washing is a safe and effective method of cleaning your hands. Use a soap and water solution to wash your hands.

⚠️ This is educational only — please consult a qualified clinician.
------------------------------------------------------



In [28]:
import nbformat

path = "/content/AlpaCare_Medical_Instruction_Assistant.ipynb"

# Load notebook
with open(path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# Remove widgets metadata if present
if "widgets" in nb["metadata"]:
    del nb["metadata"]["widgets"]

# Save cleaned notebook
cleaned_path = "/content/AlpaCare_Cleaned.ipynb"
with open(cleaned_path, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print("✅ Notebook cleaned successfully! Ready to upload to GitHub.")

✅ Notebook cleaned successfully! Ready to upload to GitHub.
